## Équipe 6 du groupe 2

- Nina Barbado, Matricule : 2161181
- Benjamin Djian, Matricule : 2226903
- Isadora Gicquel, Matricule : 2162349
- Sanmar Yared Simon, Matricule : 1938126

## <center> École Polytechnique de Montréal <br> Département Génie Informatique et Génie Logiciel <br>  INF8460 – Traitement automatique de la langue naturelle <br> </center>
## <center> TP1 INF8460 <br>  Automne 2022 </center>

## 1. DESCRIPTION
Dans ce TP, l’idée est d’effectuer de la recherche de passages de texte dans un corpus à partir d’une question en langue naturelle. Les questions et passages sont en anglais.

Voici un exemple : <br>
__Entrée : Question :__ where is the show the vikings filmed  

__Solution - Trouver un passage qui contient la réponse à la question :__ For the 2012 BBC Documentary series, see Vikings (TV documentary series) . Vikings is an Irish-Canadian historical drama television series written and created by Michael Hirst for the television channel History. Filmed in <mark>__Ireland__</mark>, it premiered on 3 March 2013 in the United States and Canada. Vikings is inspired by the sagas of Viking Ragnar Lothbrok, one of the best-known legendary Norse heroes and notorious as the scourge of England and France. The show portrays Ragnar as a former farmer who rises to fame by successful raids into England, and eventually becomes king of Denmark, with the support of his family and fellow warriors: his brother Rollo, his son Bjorn Ironside, and his wives—the shieldmaiden Lagertha and the princess Aslaug.

Ici la réponse est en gras dans le texte.

## 2. LIBRARIES PERMISES
- Jupyter notebook
- NLTK
- Numpy 
- Pandas
- Sklearn
- Pour toute autre librairie, demandez à votre chargé de laboratoire

## 3. INFRASTRUCTURE

- Vous avez accès aux GPU du local L-4818. Dans ce cas, vous devez utiliser le dossier temp (voir le tutoriel VirtualEnv.pdf)

## 4. DESCRIPTION DES DONNEES

Dans ce projet, vous utiliserez le jeu de données dans le répertoire _data_. 

Nous vous fournissons un ensemble de données qui comprend un corpus (_corpus.csv_) qui contient tous les passages et leurs identificateurs (ID) et un jeu de données qui associe une question, un passage, et une réponse qui est directement extraite du passage. 
Ce jeu de données est composé de deux sous-ensembles : 
- Train : ensemble d’entraînement de la forme <QuestionID, QuestionText, PassageID, Réponse>. Le but est donc de construire un modèle qui retrouve les N passages les plus pertinents pour répondre à la question.
- Test : Un ensemble secret qui est utilisé pour évaluer votre modèle. Il est de la forme <QuestionID, Question>. Votre système doit trouver dans le corpus _corpus.csv_ le ou les passages les plus pertinents.
- Notez que nous ne fournissons pas d’ensemble de validation car nous n’essayons pas d’entrainer un algorithme d’apprentissage machine dans ce TP.
- Notez qu’il est possible de répondre aux requis du TP sans utiliser la réponse à la question.


## 5. ETAPES DU TP 
A partir du notebook _inf8460_A22_TP2_ qui est distribué, vous devez réaliser les étapes suivantes. (Noter que les cellules dans le squelette sont là à titre informatif - il est fort probable que vous rajoutiez des sections au fur et à mesure de votre TP).

Ci-dessous définir la constante _PATH_ qui doit être utilisée par votre code pour accéder aux fichiers. Il est attendu que pour la correction, le chargé de lab n'ait qu'à changer la valeur de _PATH_ pour le répertoire où se trouver les fichiers de datasets.

In [ ]:
PATH = "C://Users/isado/NLP/TP2-20220926/TP2/data/"
PATH = "/content/"

In [ ]:
import time
import pandas as pd
import nltk
import numpy as np
import sklearn
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw-1.4')

start = time.time()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
train_init=pd.read_csv(PATH+"train.csv")
test_init=pd.read_csv(PATH+"test.csv")
corpus_init=pd.read_csv(PATH+"corpus.csv")

train=pd.read_csv(PATH+"train.csv")
test=pd.read_csv(PATH+"test.csv")
corpus=pd.read_csv(PATH+"corpus.csv")

In [ ]:
train.head(10)

,id,question,paragraph_id,answer
0,0,what type of convergence are the andes mountains,1151,Oceanic-continental convergence
1,1,What is a beret,8881,"A soft, round, flat-crowned hat, usually of wo..."
2,2,aliza name meaning,5925,Joy
3,3,how to find net realizable value of receivables,3950,Subtract the dollar amount of allowance for un...
4,4,What temperature can i put potted impatiens ou...,5841,50 to 55 degrees Fahrenheit
5,5,how much does it cost to become a notary public,7743,"$5,000"
6,6,why did the earthquake in ecuador happen,4076,The earthquake is likely to be a result of thr...
7,7,definition of wazoo,7025,A slang term for the anus.
8,8,define business proposal,10005,A business proposal is a written document sent...
9,9,aggravated assault charge definition illinois,6319,Assault is often associated with charges of ba...


In [ ]:
corpus.head(10)

,id,paragraph
0,0,&nbsp. The Maa speaking peoples of East Africa...
1,1,The Racketeer Influenced and Corrupt Organizat...
2,2,Parts: $145 - $213. The average cost for a bra...
3,3,Baroque music forms a major portion of the cla...
4,4,"Bifocals with separate lenses. Bifocals, are e..."
5,5,"The circumflex is a diacritic in the Latin, Gr..."
6,6,Some say pagan means country dweller. Unfortun...
7,7,Without crackling-sear first in a pan. Place i...
8,8,Skating with Celebrities winners Kristy Swanso...
9,9,depends on the doctor. both my kids have had t...


In [ ]:
test.head(10)

,id,question
0,10000,why was the reciprocity treaty cancelled
1,10001,average temp in adirondacks
2,10002,what kind of items did the tang trade
3,10003,what organ system does schizophrenia affect
4,10004,long service leave definition
5,10005,when was the electric light bulb invented
6,10006,cost of credit hour at university of michigan ...
7,10007,cost of landscape gravel
8,10008,how much does cost to build a garage
9,10009,what is bagel in french


### 5.1. Pré-traitement (5 points)
Les passages et questions de votre ensemble de données doivent d’abord être représentés et indexés pour ensuite pouvoir effectuer une recherche de passage pour répondre à une question. On vous demande donc d’implémenter une étape de pré-traitement des données.

Vous devez effectuer le pré-traitement du corpus (questions, passages) en convertissant le texte en minuscules, en segmentant le texte, en supprimant les mots outils et en racinisant (stemming) le texte. Votre fonction doit ensuite imprimer le nombre total de jetons dans le corpus d’entrainement (questions, passages). 

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import re
nltk.download('wordnet')
stopwords = stopwords.words('english')
stemmer = nltk.stem.porter.PorterStemmer()

def preprocess(sentence):
  """
  :param sentence: Une string qui sera pré-traitée
  :returns: La même string, après les traitements suivants 
    1) Mise en minuscule
    2) Suppression de la ponctuation
    3) Segmentation
    4) Retrait des stopwords
    5) Stemmatisation
  """
  sentence=sentence.lower()
    
  sentence=re.sub(r'[^\w\s]', ' ', sentence)
  
  tokens=word_tokenize(sentence)
  
  tokens_clean=[word for word in tokens if word not in stopwords]

  sentence_stem = [stemmer.stem(token) for token in tokens_clean] 
  
  return sentence_stem

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# On applique la fonction preprocess à nos données
train['question']=train['question'].apply(preprocess)
train['answer']=train['answer'].apply(preprocess)
test['question']=test['question'].apply(preprocess)
corpus['paragraph']=corpus['paragraph'].apply(preprocess)

In [ ]:
train.head(10)

,id,question,paragraph_id,answer
0,0,"[type, converg, and, mountain]",1151,"[ocean, continent, converg]"
1,1,[beret],8881,"[soft, round, flat, crown, hat, usual, woven, ..."
2,2,"[aliza, name, mean]",5925,[joy]
3,3,"[find, net, realiz, valu, receiv]",3950,"[subtract, dollar, amount, allow, uncollect, a..."
4,4,"[temperatur, put, pot, impatien, outdoor]",5841,"[50, 55, degre, fahrenheit]"
5,5,"[much, cost, becom, notari, public]",7743,"[5, 000]"
6,6,"[earthquak, ecuador, happen]",4076,"[earthquak, like, result, thrust, fault, caus,..."
7,7,"[definit, wazoo]",7025,"[slang, term, anu]"
8,8,"[defin, busi, propos]",10005,"[busi, propos, written, document, sent, prospe..."
9,9,"[aggrav, assault, charg, definit, illinoi]",6319,"[assault, often, associ, charg, batteri, batte..."


In [ ]:
corpus.head(10)

,id,paragraph
0,0,"[nbsp, maa, speak, peopl, east, africa, believ..."
1,1,"[racket, influenc, corrupt, organ, act, common..."
2,2,"[part, 145, 213, averag, cost, brake, pad, rot..."
3,3,"[baroqu, music, form, major, portion, classic,..."
4,4,"[bifoc, separ, lens, bifoc, eyeglass, two, dis..."
5,5,"[circumflex, diacrit, latin, greek, cyril, scr..."
6,6,"[say, pagan, mean, countri, dweller, unfortun,..."
7,7,"[without, crackl, sear, first, pan, place, pre..."
8,8,"[skate, celebr, winner, kristi, swanson, lloyd..."
9,9,"[depend, doctor, kid, tube, 1, age, 3, 1, 13m,..."


In [ ]:
test.head(10)

,id,question
0,10000,"[reciproc, treati, cancel]"
1,10001,"[averag, temp, adirondack]"
2,10002,"[kind, item, tang, trade]"
3,10003,"[organ, system, schizophrenia, affect]"
4,10004,"[long, servic, leav, definit]"
5,10005,"[electr, light, bulb, invent]"
6,10006,"[cost, credit, hour, univers, michigan, flint]"
7,10007,"[cost, landscap, gravel]"
8,10008,"[much, cost, build, garag]"
9,10009,"[bagel, french]"


In [ ]:
nbr_tokens = train['question'].str.len().sum() + corpus['paragraph'].str.len().sum()
print('Le nombre de tokens est : '+str(nbr_tokens))

Le nombre de tokens est : 500382


### 5.2. Représentation de questions et de passages (35 points)
*1) (2.5 points) Construction du vocabulaire.* À cette étape, vous devez construire un vocabulaire à partir de votre corpus d’entrainement pré-traité. Notez que les <u>questions et passages</u> de l’<u>ensemble d’entrainement</u> doivent être inclus dans la construction du vocabulaire.

In [ ]:
def build_voc(*documents):
  """
  Construit le vocabulaire d'un corpus.
  :param documents: Un itérable regroupant tous les documents sur lesquels on souhaite construire un vocabulaire
  :returns: Le vocabulaire des documents passé en entrée, sous forme de liste sans doublons.
  """
  voc = [word for document in documents for row in document for word in row] # voc contient tous les mots de toutes les lignes de tous les documents passés en paramètres
  return list(set(voc)) # Les doublons sont supprimés grâce à l'utilisation d'un set


In [ ]:
vocabulary = build_voc(train['question'], corpus['paragraph'])
print("Il y a {} mots distincts dans le vocabulaire.".format(len(vocabulary)))
print(vocabulary)

Il y a 30282 mots distincts dans le vocabulaire.
['fewster', 'louisvil', 'oxbow', 'rajini', 'hopkin', 'ere', 'gfe', 'stood', 'harbour', 'hardwood', 'tempeh', 'vector', 'perat', 'expressionist', '64th', 'ancestrydna', 'tylenol', 'cnr', 'medulla', 'noisi', 'hyperthyroid', 'terpen', 'mordant', '56000', 'phrik', 'unbeat', 'embryon', 'refract', 'mɔ', 'pet', 'disast', '174', 'buchanan', 'gōngmùhn', 'contra', 'dungeon', 'elcomet', 'czech', 'cd4', '944', 'deaneri', 'commod', '181', 'strut', 'cemil', '¼', 'herpetolog', 'temp', 'leaki', 'combo', 'goliath', 'cri', 'neuroptera', 'dignifi', 'semiarid', 'ed', 'wiki', 'neopren', 'exoplanetari', 'wufoo', 'serin', 'illus', 'ikeda', 'ritalin', 'jp', 'donkey', 'mccurri', '江珧', 'yttrium', 'saltimbocca', 'biodivers', 'eyesmart', 'lgbti', 'nsa', 'prerecord', 'mourner', 'compliant', 'uncompromis', 'grandchild', 'subito', 'seamer', 'luster', 'speedi', 'bandag', 'destitut', 'σύνθεσις', 'mycelium', 'blek', 'solidifi', 'hongqiao', 'dystonia', 'electromyographi',

*2) (10 points) Création d’un modèle sac de mots pour l’ensemble d’entrainement et de test.* En utilisant sklearn et à partir de votre corpus pré-traité, vous représentez chaque passage et question de vos deux ensembles avec votre vocabulaire, en utilisant un modèle sac de mots des n-grammes (n=1-2) qu’ils contiennent et en pondérant ces éléments avec TF-IDF.

In [ ]:
# Pour utiliser le TFIDFVectorizer de skelarn, on prétraite les données :

train_question=[" ".join(x) for x in train['question']]
train_answer=[" ".join(x) for x in train['answer']]
corpus_paragraph=[" ".join(x) for x in corpus['paragraph']]
test_question=[" ".join(x) for x in test['question']]

In [ ]:
print(test_question)
print(corpus_paragraph)

['reciproc treati cancel', 'averag temp adirondack', 'kind item tang trade', 'organ system schizophrenia affect', 'long servic leav definit', 'electr light bulb invent', 'cost credit hour univers michigan flint', 'cost landscap gravel', 'much cost build garag', 'bagel french', 'weight averag', 'cost dolli', 'incom elig medicaid', 'start phase busi definit', 'ga exchang surfac human', 'long take bodi process alcohol', 'much cost transfer ownership hous', 'radiofrequ denerv', 'pharmacist', 'obtur nerv', 'code switch occur', 'demand bill electr', 'name differ type roach', 'elev rdw blood test', 'caus peripher arteri diseas', 'lenina use soma', 'receipt must turn per diem', 'cost egg mcmuffin', 'mean donkey', 'kind characterist region', 'minimum salari wage india', 'care miniatur daffodil pot flower', 'hopi candl therapi', 'weight distribut hitch', 'drumstick', 'eleph chess call', 'e coli function dna polymeras iii answer', 'age earli intervent autism', 'regress tax', 'bodi temperatur mamm

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# On crée notre TfidfVectorizer

tfidf = TfidfVectorizer(vocabulary=vocabulary, ngram_range = (1, 2))

#On applique aux questions de l'ensemble d'entrainement

tfidf_train_question = tfidf.fit_transform(train_question).toarray()
tfidf_train_question = pd.DataFrame(tfidf_train_question, columns=tfidf.get_feature_names_out(),index=train['id'])
print(tfidf_train_question)

#On applique aux réponses de l'ensemble d'entrainement
                        
tfidf_train_answer = tfidf.fit_transform(train_answer).toarray()
tfidf_train_answer = pd.DataFrame(tfidf_train_answer, columns=tfidf.get_feature_names_out(),index=train['id'])
print(tfidf_train_answer)

#On applique au corpus

tfidf_corpus= tfidf.fit_transform(corpus_paragraph).toarray()
tfidf_corpus = pd.DataFrame(tfidf_corpus, columns=tfidf.get_feature_names_out(),index=corpus['id'])
print(tfidf_corpus)

#On applique à l'ensemble de questions test

tfidf = TfidfVectorizer(vocabulary=vocabulary, ngram_range = (1, 2))
tfidf_test_question = tfidf.fit_transform(test_question).toarray()
tfidf_test_question = pd.DataFrame(tfidf_test_question, columns=tfidf.get_feature_names_out(),index=test['id'])
print(tfidf_test_question)

      fewster  louisvil  oxbow  rajini  hopkin  ere  gfe  stood  harbour  \
id                                                                         
0         0.0       0.0    0.0     0.0     0.0  0.0  0.0    0.0      0.0   
1         0.0       0.0    0.0     0.0     0.0  0.0  0.0    0.0      0.0   
2         0.0       0.0    0.0     0.0     0.0  0.0  0.0    0.0      0.0   
3         0.0       0.0    0.0     0.0     0.0  0.0  0.0    0.0      0.0   
4         0.0       0.0    0.0     0.0     0.0  0.0  0.0    0.0      0.0   
...       ...       ...    ...     ...     ...  ...  ...    ...      ...   
9995      0.0       0.0    0.0     0.0     0.0  0.0  0.0    0.0      0.0   
9996      0.0       0.0    0.0     0.0     0.0  0.0  0.0    0.0      0.0   
9997      0.0       0.0    0.0     0.0     0.0  0.0  0.0    0.0      0.0   
9998      0.0       0.0    0.0     0.0     0.0  0.0  0.0    0.0      0.0   
9999      0.0       0.0    0.0     0.0     0.0  0.0  0.0    0.0      0.0   

      hardw

3) Maintenant que vous avez une représentation de vos passages et questions, il faut être capable de déterminer quel passage sera le plus pertinent pour la question posée

*a) (5 points)* Pour cela, vous devez écrire une fonction pour évaluer la similarité cosinus entre la représentation de la question et celle de chaque passage.

In [ ]:
def get_matrix_cosine_similarity(questions,passages):
  """
  :param questions: Un dataframe regroupant les valeurs TF IDF des questions
  :param passages: Un dataframe regroupant les valeurs TF IDF des passages
  :returns: Un tableau numpy, où chaque ligne est la similarité cosinus avec chaque passage
  """
  return sklearn.metrics.pairwise.cosine_similarity(questions,passages)

In [ ]:
matrix_cosine=get_matrix_cosine_similarity(tfidf_train_question,tfidf_corpus)
print(matrix_cosine)

*b) (5 points)* Vous devez écrire une fonction `rank_passages`, qui au moyen de la précédente, retourne pour chaque question, le top-N (N=1,5,10, 50) des passages utiles pour répondre à la question (N est un paramètre). Ces passages devront être ordonnés du plus pertinent au moins pertinent. Idéalement le passage à la position 1 sera celui qui contient la réponse à la question.

In [ ]:
def rank_passages(tfidf_question, tfidf_paragraph,matrix_cosine, N):
    
  """
  :param tfidf_question: Un dataframe regroupant les valeurs TF IDF des questions
  :param tfidf_paragraph: Un dataframe regroupant les valeurs TF IDF des passages
  :param matrix_cosine: Matrice regroupant la similarité cosinus entre les questions et les passages
  :param N: le nombre de passages voulu en sortie
  :returns: Un tableau numpy contenant les N passages les plus pertinents pour chaque question de tfidf_question
  """
  passages_top_N=np.zeros((len(tfidf_question),N), dtype = int)
    
  i=0
  for similarity in matrix_cosine:
    top_N= np.argsort(similarity)[-N:][::-1] # On trie les similarités par ordre croissant, on prend les N dernières, et on inverse cette liste
    j=0
    for n in top_N :
        passages_top_N[i,j]=n
        j+=1
    i+=1
      

  return passages_top_N

In [ ]:
topN=rank_passages(tfidf_train_question,tfidf_corpus,matrix_cosine, 4)
print(topN)

*c) (2.5 points)* Vous devez exécuter votre fonction `rank_passages` sur toutes les questions de l’ensemble d'entraînement et sauvegarder vos résultats pour l’ensemble d’entrainement selon le même format que _passage_submission.csv_ sous le nom _train_passage_submission.csv_. 

In [ ]:
def generate_file(questions, corpus, filename, passages=None):
  """
  :param tfidf_question: Un dataframe regroupant les valeurs TF IDF des questions
  :param corpus: Un dataframe regroupant les valeurs TF IDF des passages
  :param filename: Une string désignant le nom du fichier en sortie, sans l'extension .csv
  :param passages: Un dataset regroupant les gold standards label, c'est-à-dire les passages considérés comme justes. Si on ne le précise pas alors la colonne correspondante du csv sera vide
  :returns: Rien, mais génère un fichier csv à l'emplacement PATH + filename sous le même format que passage_submission.csv
  """
  results = []
  similarity = get_matrix_cosine_similarity(questions, corpus) # On calcule la similarité, c'est l'étape qui prend le plus de temps à s'exécuter
  ranks = rank_passages(questions, corpus,similarity, 50) # On trouve les 50 passages les plus probables pour les questions, avec rank_passages
  rank_index = 0

  for rank in ranks:
    true_passage = passages['paragraph_id'][rank_index] if passages is not None else '' # Si on ne précise pas le paramètre passage, la colonne 'gold standard passage' est laissé vide
    rank_index += 1
    top_1 = str(rank[0])
    top_5 = ';'.join([str(item) for item in rank[0:5]])
    top_10 = ';'.join([str(item) for item in rank[0:10]])
    top_50 = ';'.join([str(item) for item in rank])

    results.append([true_passage, top_1, top_5, top_10, top_50])

  pd.DataFrame(results,  index = questions.index).to_csv(PATH + filename + '.csv', header = ['gold standard passage', 'top_1', 'top_5', 'top_10', 'top_50'], sep=';') # On utilise sep=';' pour garantir que le fichier s'ouvre correctement sous excel



In [ ]:
generate_file(questions = tfidf_train_question, corpus = tfidf_corpus, filename='train_passage_submission', passages = train)

In [ ]:
train_passage_submission=pd.read_csv(PATH+"train_passage_submission.csv",sep=';')
print(train_passage_submission)

*d) (2.5 points)* Vous devez écrire une fonction `show_top_n_passages` qui affiche le top-5 ainsi que le paragraphe attendu pour une question donnée. 

In [ ]:
def show_top_n_passages(id_question, train_passage_submission ):
  """
  :param id_question: L'identifiant unique de la question et le fichier train_passage_submission avec les topN pour chaque question
  :param train_passage_submission: Le dataframe correspondant à la lecture du csv contenant les top-N des gold standards
  :returns: Rien, mais affiche le paragraphe attendu en réponse à la question qui a pour ID question_id, et la liste ordonnée des indices des 5 passages les plus probables
  """
  true_pass = train_passage_submission.loc[id_question,'gold standard passage']
  top_5 = train_passage_submission.loc[id_question,'top_5']
  print('Paragraphe attendu pour la question '+str(id_question)+' :')
  print(true_pass)
  print('Top_5 estimé pour la question '+str(id_question)+' :')
  print(top_5)


In [ ]:
#On teste pour la question 100
show_top_n_passages(100,train_passage_submission)

NameError: ignored

_e) (5 points) Évaluation:_ En utilisant votre ensemble d’entrainement, vous devez implémenter une fonction `evaluate` qui, à partir du fichier *train_passage_submission.csv* et de la comparaison entre la colonne *Gold standard passage* et vos top-N, calcule la précision top-N (N=1,5,10,50) sur l’ensemble de données en paramètre et l’afficher. Notez que nous réutiliserons la même fonction pour l’évaluation de votre ensemble de test, veillez donc à ce que la fonction soit générique et respecte le format du fichier.

In [ ]:
def evaluate(filename, N):
  """
  :param filename: Le nom du fichier csv à évaluer (sans l'extension .csv)
  :param N: La précision voulue pour l'évaluation. Pour la précision usuelle, utiliser 1. Les valeurs acceptées sont 1, 5, 10 et 50.
  :returns: Un score de précision top-N du fichier nommé filename
  """
  if N not in [1, 5, 10, 50]:
    raise Exception('Le paramètre N doit être égal à 1, 5, 10 ou 50. N vaut actuellement {}'.format(N))
  
  score = 0
  data = pd.read_csv(PATH + filename + '.csv', sep=';')

  for index in data.index.to_list(): # On parcourt tous les index du fichier
        if str(data.loc[index]['gold standard passage']) in str(data.loc[index]['top_'+str(N)]).split(';'): # Si le gold standard passage est dans le top N, on incrémente le score
          score += 1

  return score/len(data)

In [ ]:
print(evaluate('train_passage_submission', 50))
print(evaluate('train_passage_submission', 1))

1.0
0.6666666666666666


f) Discussion : Observez les paragraphes obtenus au hasard sur un échantillon de questions. Lorsque le premier passage n’est pas celui qui est attendu (qui contient la réponse), pouvez-vous donner une raison pour sa haute similarité avec la question ? Quels sont les facteurs qui peuvent expliquer, globalement, les erreurs du modèle ? Etayez votre discussion avec des exemples précis.

In [ ]:
train_passage_submission=pd.read_csv(PATH+'train_passage_submission.csv', sep=';')
#on sélectionne les prévisions dont le premier passage n'est pas celui attendu 
train_passage_submission_False=train_passage_submission[train_passage_submission['gold standard passage']!=train_passage_submission['top_1']]

#On crée la liste des identifiants des questions dont la première prévision n'est pas celle attendu
train_passage_submission_Index_Question=train_passage_submission_False.index.to_list()

#On crée la liste des identifiants des réponses des questions dont la première prévision n'est pas celle attendu
train_passage_submission_True_Para=train_passage_submission_False['gold standard passage'].to_list()

#On crée la liste des identifiants des paragraphes estimés
train_passage_submission_False_Para=train_passage_submission_False['top_1'].to_list()

print(train_passage_submission_False)
#On affiche les 5 premiers
for i in range(5):
  index_question=train_passage_submission_Index_Question[i]
  index_true_para=train_passage_submission_True_Para[i]
  index_false_para=train_passage_submission_False_Para[i]


  print('La question est :')
  print(train.loc[index_question,'question'])
  print('La vraie réponse est :')
  print(corpus.loc[index_true_para,'paragraph'])
  print('La réponse estimée en top_1 est :')
  print(corpus.loc[index_false_para,'paragraph'])

   Unnamed: 0  gold standard passage  top_1                      top_5  \
0           0                   1151  10831  10831;1151;5138;8541;1280   
5           5                   7743   9691    9691;7743;191;5368;4682   

                                              top_10  \
0  10831;1151;5138;8541;1280;6236;4110;5821;9136;...   
5  9691;7743;191;5368;4682;6615;7545;1256;2390;10676   

                                              top_50  
0  10831;1151;5138;8541;1280;6236;4110;5821;9136;...  
5  9691;7743;191;5368;4682;6615;7545;1256;2390;10...  
La question est :
['type', 'converg', 'and', 'mountain']
La vraie réponse est :
['answer', 'commun', 'make', 'world', 'better', 'one', 'answer', 'time', 'type', 'boundari', 'and', 'mountain', 'south', 'america', 'converg', 'plate', 'boundari', 'form', 'collis', 'south', 'american', 'plate', 'boundari', 'nazca', 'plate', 'plate', 'move', 'toward', 'specif', 'type', 'plate', 'collis', 'caus', 'cascad', 'es', 'ocean', 'continent', 'converg', 

On peut observer que pour la première question affichée, sur les quatres mots composants cette question, 2 sont présents dans la réponse estimée à deux reprises : 'and' et 'mountain'. Les contextes entre la réponse estimée et la vraie réponse semblent similaires ce qui explique une grande similarité. Dans la deuxième question affichée si on prend en compte les mots de la question sur les 5, 2 sont présents dans le top_1. Le mot 'notari' est présent 6 fois et le mot 'public' 4 fois. La haute similarité est de nouveau justifiée. 

Le calcul de similarité ne prend pas bien en compte la différence entre un même mot qui apparaît plusieurs fois dans la réponse et plusieurs mots différents qui apparaissent dans la réponse. Idf permet de diminuer l'importance des mots fréquents dans l'ensemble du corpus mais chaque paragraphe peut correspondre à un contexte particulier et un mot rare dans l'ensemble du corpus peut apparaître plusieurs fois dans un même paragraphe. 

Les facteurs qui peuvent expliquer les erreurs du modèle sont le choix de la pondération et le choix du modèle de calcul de similarité. Pour améliorer notre modèle il faudrait donc modifier la métrique tf-idf et/ou utiliser une autre méthode de calcul de similarité avec l'apprentissage machine par exemple. 

### 5.3. Exécution sur l’ensemble de test (5 points)
Vous devez exécuter la fonction `rank_passages` sur toutes les questions de *l’ensemble de test*. A cette étape, vous devez produire un fichier `passage_submission.csv` qui contient pour toutes les questions de l’ensemble de test le top-N des passages retournés par votre modèle pour y répondre. Le fichier doit respecter le format démontré dans `sample_passage_submission.csv`.

In [ ]:
generate_file(questions = tfidf_test_question, corpus = tfidf_corpus, filename='passage_submission')

### 5.4. Amélioration (10 points)
Proposez une autre métrique que TF-IDF qui permette d’améliorer les performances obtenues sur l’ensemble d’entrainement. Effectuez une recherche sur Google scholar pour vous donner des idées et indiquez vos références. Démontrez l’amélioration par une exécution de la métrique et un affichage de ses performances comparé à TF-IDF sur l’ensemble d’entrainement. Enfin, vous devez générer un fichier _test_passage_submission_improved.csv_ selon le même format précédent avec vos nouvelles réponses sur l'_ensemble de test_ selon cette nouvelle métrique. 

**Réponse** : Nous avons choisi d'utiliser une nouvelle pondération, BM-25, ou Okapi, plus précise que TF-IDF car elle prend en compte la longueur des documents utilisés.
Pour cela nous nous sommes basés sur l'article de Vincent Claveau de l'IRISA-CNRS, trouvable à l'adresse suivante : https://aclanthology.org/F12-2007.pdf

La formule que nous utilisons se trouve à la 5ème page du PDF (page numérotée 89 dans le document).
La mesure de distance est toujours la similarité cosinus.

In [ ]:
from math import log


def get_idf_okapi(vocabulary, nbr_doc, documents):
    idf_list = []
    for doc in documents:
      for token in vocabulary:
        idf = log(nbr_doc/((doc.count(token)+1)))
      idf_list.append(log((nbr_doc - idf + 0.5)/(idf+0.5)))
    
    return idf_list


def tf_idf_okapi_matrix(corpus, vocabulary, k=1.5, b=0.75):
  
  nbr_doc = len(corpus)
  average_length = sum([len(doc) for doc in corpus])/nbr_doc
  tf_idf_matrix = np.zeros((nbr_doc, len(vocabulary)))

  idf_okapi = get_idf_okapi(vocabulary, nbr_doc, corpus)

  for d, doc in enumerate(corpus):
    len_doc = len(doc)
    for t, token in enumerate(vocabulary):
      tf = doc.count(token)/len_doc
      tf_okapi = (tf*(k+1))/(tf+k*(1-b+b*len_doc/average_length))
      tf_idf_matrix[d][t] = idf_okapi[d]*tf_okapi

  return tf_idf_matrix

Création des matrices TF-IDF BM-25 :

In [ ]:
tfidf_okapi_corpus = tf_idf_okapi_matrix(corpus['paragraph'], vocabulary)
print(tfidf_okapi_corpus)

searching idf


KeyboardInterrupt: ignored

In [ ]:
tfidf_okapi_train_question = tf_idf_okapi_matrix(train['question'], vocabulary)
print(tfidf_okapi_train_question)

Création du fichier train_passage_submission_improved.csv :

In [ ]:
generate_file(questions = tfidf_okapi_train_question, filename='train_passage_submission_improved', passages = tfidf_okapi_corpus)

Comparaison des résultats avec la fonction `evaluate` :

In [ ]:
print(evaluate('train_passage_submission_improved', 50))
print(evaluate('train_passage_submission_improved', 1))

Création du fichier à soumettre pour l'ensemble de test :

In [ ]:
tfidf_okapi_test_question = tf_idf_okapi(test['question'], vocabulary)
generate_file(questions = tfidf_okapi_test_question, filename='test_passage_submission_improved', passages = tfidf_okapi_corpus)

## LIVRABLES
Vous devez remettre sur Moodle un zip contenant les fichiers suivants :
1. _Le code :_ Vous devez compléter le squelette _inf8460_A22_TP2.ipynb_ sous le nom _matricule1_matricule2_matricule3_TP1.ipynb_. Ce notebook reprend les différentes questions, et doit contenir les fonctionnalités requises avec des commentaires appropriés. Le code doit être exécutable sans erreur et accompagné des commentaires appropriés dans le notebook de manière à expliquer les différentes fonctions. Les critères de qualité tels que la lisibilité du code et des commentaires sont importants. Tout votre code et vos résultats doivent être exécutables et reproductibles ; 
2. Un fichier html représentant votre notebook complètement exécuté sous format html
3. Les fichiers de soumission de données d’entrainement et de test : _train_passage_submission_, _test_passage_submission.csv_ et _test_passage_submission_improved.csv_ ;
4. Un document _contributions.txt_ : Décrivez brièvement le pourcentage de contribution et la contribution de chaque membre de l’équipe. Tous les membres sont censés contribuer au développement. Bien que chaque membre puisse effectuer différentes tâches, vous devez vous efforcer d’obtenir une répartition égale du travail.


## EVALUATION 
Votre TP sera évalué selon les critères suivants :
1. Exécution correcte du code
2. Performance correcte des modèles
3. Organisation du notebook
4. Qualité du code (noms significatifs, structure, performance, gestion d’exception, etc.)
5. Commentaires clairs et informatifs

## CODE D’HONNEUR
- Règle 1:  Le plagiat de code est bien évidemment interdit.
- Règle 2: Vous êtes libres de discuter des idées et des détails de mise en œuvre avec d'autres équipes. Cependant, vous ne pouvez en aucun cas consulter le code d'une autre équipe INF8460, ou incorporer leur code dans votre TP.
- Règle 3:  Vous ne pouvez pas partager votre code publiquement (par exemple, dans un dépôt GitHub public) tant que le cours n'est pas fini.


In [ ]:
end = time.time()
print(f"Durée d'exécution: {(end-start)//60:.0f}min {(end-start)%60:.0f}s")